In [2]:
import matchzoo as mz

/usr/local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
print(mz.__version__)

2.2.0


In [6]:
task = mz.tasks.Ranking()
print(task)

Ranking Task


In [8]:
train_raw = mz.datasets.wiki_qa.load_data('train', task='ranking')
test_raw = mz.datasets.wiki_qa.load_data('test', task='ranking')

In [9]:
type(train_raw)

matchzoo.data_pack.data_pack.DataPack

In [10]:
#DataPack is a MatchZoo native data structure that most MatchZoo data handling processes build upon. A DataPack is consists of three pandas.DataFrame:
train_raw.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [11]:
train_raw.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [12]:
train_raw.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0
1,Q1,D1-1,0
2,Q1,D1-2,0
3,Q1,D1-3,1
4,Q1,D1-4,0


In [13]:
# It is also possible to convert a DataPack into a single pandas.DataFrame that holds all information.
train_raw.frame().head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0


# Preprocessing

In [14]:
preprocessor = mz.preprocessors.BasicPreprocessor()

In [15]:
type(preprocessor)

matchzoo.preprocessors.basic_preprocessor.BasicPreprocessor

In [16]:
preprocessor.fit(train_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5847.76it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4061.77it/s]
Building Vocabulary from a datapack.: 100%|██████████| 473697/473697 [00:00<00:00, 2192634.26it/s]


In [17]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x10ba611d0>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x13bb22890>,
 'vocab_size': 17483,
 'embedding_input_dim': 17483,
 'input_shapes': [(30,), (30,)]}

In [18]:
train_processed = preprocessor.transform(train_raw)
test_processed = preprocessor.transform(test_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5612.92it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:05<00:00, 3641.10it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 66836.96it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 633/633 [00:00<00:00, 6924.16it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 5961/5961 [00:01<00:00, 3999.79it/s]
Processing text_right with transform: 100%|██████████| 5961/5961 [00:00<00:00, 61240.84it/s]


In [19]:
train_processed.left.head()

,text_left,length_left
id_left,,
Q1,"[4788, 1519, 6202, 1244, 9620, 2, 0, 0, 0, 0, ...",6
Q2,"[4788, 1519, 11003, 4797, 7100, 11003, 1000, 1...",15
Q5,"[4788, 1934, 12577, 17170, 13606, 0, 0, 0, 0, ...",5
Q6,"[4788, 3526, 7503, 11003, 8694, 9053, 14612, 6...",8
Q7,"[4788, 7888, 7552, 777, 2371, 2241, 3126, 298,...",8


In [28]:
sequence = train_processed.left.loc['Q1']['text_left']
print('Original Text:', sequence)

Original Text: [4788, 1519, 6202, 1244, 9620, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [29]:
print('Text with padding: \n', '_'.join([preprocessor.context['vocab_unit'].state['index_term'][i] for i in sequence]))

##Text with padding: 
##how_are_glacier_caves_formed__<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>

Text with padding: 
 how_are_glacier_caves_formed__<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>_<PAD>


# Build Model

In [30]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM,
 matchzoo.contrib.models.match_srnn.MatchSRNN,
 matchzoo.contrib.models.hbmp.HBMP,
 matchzoo.contrib.models.esim.ESIM,
 matchzoo.contrib.models.bimpm.BiMPM,
 matchzoo.contrib.models.diin.DIIN,
 matchzoo.models.conv_knrm.ConvKNRM]